In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import os
import requests
import itertools

from tqdm import tqdm
%matplotlib inline

face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_eye.xml')

In [5]:
def try_get_people_face(links):
    num_images = 0
    
    for link in tqdm(links):
        try:
            raw_image = requests.get(link)
            raw_image_transf = np.fromstring(raw_image.content, dtype='uint8')

            cv2_image = cv2.imdecode(raw_image_transf, cv2.IMREAD_UNCHANGED)
            cv2_image_resized = cv2.resize(cv2_image, (300, 300)) 
            cv2_image_resized_without_eyes = cv2_image_resized.copy()

            cv2_image_resized_gray = cv2.cvtColor(cv2_image_resized, 0)
            faces = face_cascade.detectMultiScale(cv2_image_resized_gray, 1.3, 5)

            accept = False
            for (x, y, w, h) in faces:
                face_gray = cv2_image_resized_gray[y:y+h, x:x+w]

                eyes = eye_cascade.detectMultiScale(face_gray)
                for (ex, ey, ew, eh) in eyes:
                    accept = True
                    cv2_image_resized_without_eyes[y+ey:y+ey+eh, x+ex:x+ex+ew] = 0

            if accept:
                num_images += 1
                cv2.imwrite('data/train/p_'+str(num_images)+'.jpg', cv2_image_resized)
                cv2.imwrite('data/train/p_'+str(num_images)+'_without_eyes.jpg', cv2_image_resized_without_eyes)

        except Exception as e:
            pass
        
    print('Downloaded {} faces'.format(num_images))

In [ ]:
urls = [
    'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n09618957',
    'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n07942152'
]
    
links = []
for url in urls:
    response = requests.get(url)
    links.append([link[:-1] for link in response.text.split('\n')])

links = list(itertools.chain(*links))
try_get_people_face(links)

  3%|▎         | 81/3005 [01:23<50:11,  1.03s/it]  